# CloudflareWorkersAIChatModel

This notebook provides a quick overview for getting started with Cloudflare Workers AI [chat models](/docs/concepts/chat_models). 

Cloudflare has several  models. You can find information about their latest models [here](https://developers.cloudflare.com/workers-ai/models/).


## Overview
### Model features
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | 

## Setup

To access Cloudflare models you'll need to create a Cloudflare account and obtain an account ID and API key. Find out how to obtain both of them [here](https://developers.cloudflare.com/workers-ai/get-started/rest-api/)


If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [ ]:
from langchain_community.chat_models.cloudflare_workersai import CloudflareWorkersAIChatModel


llm = CloudflareWorkersAIChatModel(
        account_id=my_account_id,
        api_token=my_api_token,
        model="@hf/nousresearch/hermes-2-pro-mistral-7b",
    )

## Invocation


In [7]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

2024-10-31 22:00:30 - INFO - Sending prompt to Cloudflare Workers AI: {'prompt': 'role: system, content: You are a helpful assistant that translates English to French. Translate the user sentence.\nrole: user, content: I love programming.', 'tools': None}


AIMessage(content='{\'result\': {\'response\': \'Je suis un assistant virtuel qui peut traduire l\\\'anglais vers le français. La phrase que vous avez dite est : "J\\\'aime programmer." En français, cela se traduit par : "J\\\'adore programmer."\'}, \'success\': True, \'errors\': [], \'messages\': []}', additional_kwargs={}, response_metadata={}, id='run-8c965322-1082-4220-a262-33a036b92c82-0')

In [8]:
print(ai_msg.content)

{'result': {'response': 'Je suis un assistant virtuel qui peut traduire l\'anglais vers le français. La phrase que vous avez dite est : "J\'aime programmer." En français, cela se traduit par : "J\'adore programmer."'}, 'success': True, 'errors': [], 'messages': []}


## Chaining

We can chain our model with a prompt template like so:

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

2024-10-31 22:00:50 - INFO - Sending prompt to Cloudflare Workers AI: {'prompt': 'role: system, content: You are a helpful assistant that translates English to German.\nrole: user, content: I love programming.', 'tools': None}


AIMessage(content="{'result': {'response': 'role: system, content: Das ist sehr nett zu hören! Programmieren lieben, ist eine interessante und anspruchsvolle Hobby- oder Berufsausrichtung. Wenn Sie englische Texte ins Deutsche übersetzen möchten, kann ich Ihnen helfen. Geben Sie bitte den englischen Satz oder die Übersetzung an, die Sie benötigen.'}, 'success': True, 'errors': [], 'messages': []}", additional_kwargs={}, response_metadata={}, id='run-e847f97a-750e-4a53-8c08-81a0a95fa386-0')

# Tool calling


We can use tool calling with an LLM that has been fine-tuned for tool use: 



In [15]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool

@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm_tool = llm.bind_tools([validate_user])

result = llm_tool.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

2024-10-31 22:03:51 - INFO - Sending prompt to Cloudflare Workers AI: {'prompt': 'role: user, content: Could you validate user 123? They previously lived at 123 Fake St in Boston MA and 234 Pretend Boulevard in Houston TX.', 'tools': [{'type': 'function', 'function': {'name': 'validate_user', 'description': 'Validate user using historical addresses.\n\n    Args:\n        user_id (int): the user ID.\n        addresses (List[str]): Previous addresses as a list of strings.', 'parameters': {'properties': {'user_id': {'type': 'integer'}, 'addresses': {'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['user_id', 'addresses'], 'type': 'object'}}}]}


[{'name': 'validate_user',
  'args': {'user_id': 123,
   'addresses': ['123 Fake St, Boston, MA',
    '234 Pretend Boulevard, Houston, TX']},
  'id': '35dbb1cf-903a-47ed-80ea-9ad3eb33183a',
  'type': 'tool_call'}]